### Introdução ao uso de LangChain

O Langchain permite trabalhar facilmente com diversos modelos.

**Modelos:**

- [Documentação LangChain](https://python.langchain.com/docs/integrations/llms/)

### Instalação

In [1]:
import torch
import os
import getpass # Para informar o token do huggingFace

# Transformers do HuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, AwqConfig

# Integrando Langchain juntamente com o Hugging Face
from langchain_huggingface import HuggingFacePipeline

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate
)

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv

/home/wolf/anaconda3/envs/spero-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-22 09:10:25.161037: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-22 09:10:29.405914: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-22 09:10:32.604851: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slight

AttributeError: _ARRAY_API not found

In [2]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

### Carregando LLM via pipeline (a mesma da aula anterior)

**Utilizando Langchain juntamente com o HuggingFace**

Aqui iremos utilizar o mesmo código e modelo da aula anterior, porém, utilizando o Langchain para pré processar os textos, sem a necessidade de criar templates como fizemos nas aulas com HuggingFace.

In [3]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, # Modelo que será baixado
    device_map = "cuda", # Deve ser carregada em uma GPU habilitada para CUDA
    torch_dtype = "auto",
    trust_remote_code = False,
    attn_implementation="eager" # Método de implementação para o mecanismo de atenção
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.65s/it]


A temperatura controla a aleatoriedade do processo de geração de texto.

Valores mais baixos tornam a saída do modelo mais determinística, enquanto que valores mais altos aumentam a "criatividade" do algoritimo (algo que também pode causar a alucinação).

In [5]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500, # Será gerado no máximo 500 tokens
    do_sample = True, # Adiciona fator de aleatoriedade na geração dos textos
    repetition_penalty = 1.1, # Desencoraja o modelo a gerar textos ou frases muito repetitivas. 1 é padrão (nenhuma penalidade)
    return_full_text = False # Determina se deve ser retornado o texto completo, incluindo o prompt completo ou apenas o texto gerado
)

Device set to use cuda


In [6]:
llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
output = llm.invoke(input="Qual é o sentido da vida?")
print(output)



### Response:O significado ou propósito da vida tem sido objeto de debate filosófico, espiritual e científico por séculos. A resposta a essa pergunta pode variar muito dependendo das crenças pessoais, culturais e religiosas individuais. Alguns argumentam que cada indivíduo deve encontrar seu próprio significado na vida através do auto-desenvolvimento, contribuição para os outros e busca de experiências gratificantes. Outros sugerem que um senso coletivo de bem estar social, harmonia comunitária e responsabilidade ambiental são aspectos importantes no contexto mais amplo da existência humana. Em última análise, muitos concordam que a busca pelo entendimento sobre o significado da vida é uma jornada contínua em vez de um destino final.


### Outros modelos Open Source (Llama)

- [Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)

Com a máquina utilizada para fazer o curso, não foi possível baixar e utilizar o modelo llama a partir do HuggingFace nem com a versão quantizada, iremos continuar utilizando o modelo *Phi-3*.

In [8]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

### Modelos de Chat

In [8]:
from langchain_core.messages import (HumanMessage, SystemMessage)
from langchain_huggingface import ChatHuggingFace

#### Adequando um modelo

Aqui estamos definindo como o modelo irá se comportar. Diferentemente dos exemplos onde precisavamos definir as tags de finalização dos textos e quem era o ator (role) manualmente, desta vez podemos definir com as classes *SystemMessage* e *HumanMessage*.

In [9]:
msgs = [
    SystemMessage(content="Você é um assistente e está respondendo perguntas gerais."),
    HumanMessage(content="Explique para mim brevemente o que seria a vida.")
]

In [10]:
chat_model = ChatHuggingFace(llm=llm) # Aqui é definido qual LLM será utilizada, no momento, estamos utilizando o Phi-3

Abaixo é o template do modelo.

In [11]:
model_template = tokenizer.chat_template
model_template

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

Abaixo é o template preenchido com os prompts.

In [12]:
chat_model._to_chat_prompt(msgs)

'<|system|>\nVocê é um assistente e está respondendo perguntas gerais.<|end|>\n<|user|>\nExplique para mim brevemente o que seria a vida.<|end|>\n<|assistant|>\n'

In [13]:
resp = chat_model.invoke(msgs)
print(resp.content)

 A vida, em sua essência mais simples, refere-se à existência de seres vivos - organismos capazes de crescer, se reproduzir, responder aos estímulos do ambiente e manter uma ordem interna através da homeostase. É caracterizada por processos biológicos complexos como metabolismo, genética e adaptação ao longo das gerações. Em termos filosóficos ou espirituais, pode abranger também as experiências humanas, incluindo consciência, emoção e autoconsciência.


A vida não apenas existe no nível físico com células vibrantes dentro dos corpos; ela transcende até envolver aspectos culturais, sociais e pessoais que definem quem somos e como interagimos entre si na sociedade humana. Ela é marcada pela busca pelo bem-estar, desenvolvimento individual e coletivo, além da contribuição para comunidades maiores e legados duradouros.


### Prompt Templates

Os modelos de prompt (promt templates) ajudam a traduzir a entrada e os parâmetros do usuário em instruções para um modelo de linguagem. Pode ser usado para orientar a resposta de um modelo, ajudando-o a entender o contexto e gerar uma saída relevante. Também facilita a criação de prompts de maneiras variáveis.

Existem alguns tipos diferentes de modelos de prompt:

#### String PromptTemplates

Esses modelos de prompt são usados para formatar uma única string e geralmente são usados para entradas mais simples.

Tipo de template utilizada para enviar uma string como parâmetro.

In [14]:
prompt_template = PromptTemplate.from_template("Escreva um poema sobre {topic}")
prompt_template.invoke({"topic": "abacates"})

StringPromptValue(text='Escreva um poema sobre abacates')

In [15]:
prompt_template

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Escreva um poema sobre {topic}')

#### ChatPromptTemplates

São usados para formatar uma lista de mensagens. Ou seja, uma lista de templates.

In [16]:
prompt = ChatPromptTemplate.from_messages({
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", "Explique-me em 1 parâgrafo o conceito de {topic}")
})

prompt.invoke({"topic": "IA"})

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente e está respondendo perguntas gerais.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explique-me em 1 parâgrafo o conceito de IA', additional_kwargs={}, response_metadata={})])